In [1]:
# Example taken from: https://www.callcentrehelper.com/erlang-c-formula-example-121281.htm
# Requirement: Find the number of agents required to manage call center transactions
# under the following parameters:
# Number of calls: 100
# In a period of minutes: 30
# Average Handling Time (seconds): 180 (3 minutes)
# Required Service Level: 80%
# Target Answer Time (Seconds): 20
# Maximum Occupancy: 85%
# Shrinkage: 30%


In [2]:
from pyworkforce.queuing import ErlangC
import pandas as pd

In [3]:
erlang = ErlangC(transactions=37906, aht=9.2, interval=15660, asa=30, shrinkage=0.30)

In [4]:
positions_requirements = erlang.required_positions(service_level=0.65, max_occupancy=0.85)
print("positions_requirements: ", positions_requirements)

positions_requirements:  {'raw_positions': 27, 'positions': 39, 'service_level': 0.9999999500072593, 'occupancy': 0.8247840688709144, 'waiting_probability': 0.25038188050012045}


In [5]:
achieved_service_level = erlang.service_level(positions=positions_requirements['raw_positions'])
print("achieved_service_level: ", achieved_service_level)

achieved_service_level:  0.9999999500072593


In [6]:
achieved_service_level = erlang.service_level(positions=positions_requirements['positions'],
                                              scale_positions=True)
print("achieved_service_level: ", achieved_service_level)

achieved_service_level:  0.9999999500072593


In [7]:
waiting_probability = erlang.waiting_probability(positions=positions_requirements['raw_positions'])
print("waiting_probability: ", waiting_probability)

waiting_probability:  0.25038188050012045


In [8]:
achieved_occupancy = erlang.achieved_occupancy(positions=positions_requirements['raw_positions'])
print("achieved_occupancy: ", achieved_occupancy)

achieved_occupancy:  0.8247840688709144


In [9]:
from pyworkforce.scheduling import MinAbsDifference

In [21]:
# Columns are an hour of the day, rows are the days
required_resources = [
    [9, 11, 17, 9, 7, 12, 5, 11, 8, 9, 18, 17, 8, 12, 16, 8, 7, 12, 11, 10, 13, 19, 16, 7],
    [13, 13, 12, 15, 18, 20, 13, 16, 17, 8, 13, 11, 6, 19, 11, 20, 19, 17, 10, 13, 14, 23, 16, 8]
]

In [22]:
# required_resources = pd.DataFrame(required_resources)
# required_resources

In [23]:
# Each entry of a shift, is an hour of the day (24 columns)
# E_sp
shifts_coverage = {"Morning": [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   "Afternoon": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
                   "Night": [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
                   "Mixed": [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]}

In [24]:
# shifts_coverage = pd.DataFrame(shifts_coverage)
# shifts_coverage

In [25]:
scheduler = MinAbsDifference(num_days=2,  # S
                             periods=24,  # P
                             shifts_coverage=shifts_coverage,
                             required_resources=required_resources,
                             max_period_concurrency=27,  # gamma
                             max_shift_concurrency=25)   # beta

In [26]:
solution = scheduler.solve()
print(solution)

{'status': 'OPTIMAL', 'cost': 157.0, 'resources_shifts': [{'day': 0, 'shift': 'Morning', 'resources': 11}, {'day': 0, 'shift': 'Afternoon', 'resources': 11}, {'day': 0, 'shift': 'Night', 'resources': 11}, {'day': 0, 'shift': 'Mixed', 'resources': 1}, {'day': 1, 'shift': 'Morning', 'resources': 13}, {'day': 1, 'shift': 'Afternoon', 'resources': 14}, {'day': 1, 'shift': 'Night', 'resources': 13}, {'day': 1, 'shift': 'Mixed', 'resources': 0}]}


In [27]:
from pyworkforce.scheduling import MinRequiredResources

In [28]:
# Columns are an hour of the day, rows are the days
required_resources = [
    [9, 11, 17, 9, 7, 12, 5, 11, 8, 9, 18, 17, 8, 12, 16, 8, 7, 12, 11, 10, 13, 19, 16, 7],
    [13, 13, 12, 15, 18, 20, 13, 16, 17, 8, 13, 11, 6, 19, 11, 20, 19, 17, 10, 13, 14, 23, 16, 8]
]

In [29]:
# Each entry of a shift, is an hour of the day (24 columns)
shifts_coverage = {"Morning": [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   "Afternoon": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
                   "Night": [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
                   "Mixed": [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]}

In [30]:
# The cost of shifting a resource if each shift, if present, solver will minimize the total cost
cost_dict = {"Morning": 8, "Afternoon": 8, "Night": 10, "Mixed": 7}

In [31]:
scheduler = MinRequiredResources(num_days=2,
                                 periods=24,
                                 shifts_coverage=shifts_coverage,
                                 required_resources=required_resources,
                                 cost_dict=cost_dict,
                                 max_period_concurrency=25,
                                 max_shift_concurrency=25)

In [32]:
solution = scheduler.solve()

print(solution)

{'status': 'OPTIMAL', 'cost': 979.0, 'resources_shifts': [{'day': 0, 'shift': 'Morning', 'resources': 12}, {'day': 0, 'shift': 'Afternoon', 'resources': 13}, {'day': 0, 'shift': 'Night', 'resources': 19}, {'day': 0, 'shift': 'Mixed', 'resources': 6}, {'day': 1, 'shift': 'Morning', 'resources': 20}, {'day': 1, 'shift': 'Afternoon', 'resources': 17}, {'day': 1, 'shift': 'Night', 'resources': 23}, {'day': 1, 'shift': 'Mixed', 'resources': 3}]}
